<a href="https://colab.research.google.com/github/ITU-Business-Analytics-Team/Business_Analytics_for_Professionals/blob/main/Part%20I%20%3A%20Methods%20%26%20Technologies%20for%20Business%20Analytics/Chapter%209%3A%20Prescriptive%20Analytics%3A%20Optimization%20and%20Modelling/9_3_Production_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prescriptive Analytics: Optimization and Modelling**
## Integer Programming

### Production Planning

In this example, the production planning of ATK-White company is conducted. ATK-White has plans to produce five new products. In Table below, the initial investment costs of the company for the products and the net profit values of the products are given. ATK-White company should also consider the following conditions.
Up to three products will be produced.

*   In order to produce product 3 or product 4, at least one of product 1 or product 2 must be produced.
*   Required labor hours and raw material for the products are given in in Table below. The company has 7000 working hours and 7000 units of raw materials during the planning period.
*   In accordance with the agreement of the company with a customer, 500 units of the 1st and 3rd products in total or 750 units of the 2nd and 4th products in total should be send to the customer.
*   If the 5th product is produced more than 500, the 4th product must be produced at most 500.
An integer programming model must be formulated that will maximize ATK-White's overall profits, taking into account the given conditions.

Products| 1 | 2 | 3 | 4 | 5
----|---|-----|-----|----|---
Investment Cost|\$15000|\$35000|\$50000|\$25000|\$40000
Unit Net Profit|\$50|\$80|\$70|\$85|\$90
Required Labor Hours per Product|5|3|7|4|5
Required Raw Material per Product|4|4|5|2|3

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

     |████████████████████████████████| 9.1 MB 5.3 MB/s 
     |████████████████████████████████| 49 kB 4.4 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

In [ ]:
#import library
import pyomo.environ as pyo
import numpy as np

# Decision Variables 
$X_i$= Production quantity for product $i$ for $i=1,2,3,4,5$

$Y_i=\begin{cases}
    1, & \text{if investment is made in product $i$ for $i=1,2,3,4,5$} .\\
    0, & \text{otherwise}.
  \end{cases}$



In [ ]:
# Numbers of products
N = 5
products = np.arange(N)

#unit profit for each prodction
profit=np.array([50,80,70,85,90])

#investment cost for each prodction
investment_cost=np.array([15000,35000,50000,25000,40000])

#Labor hour consumption of products for the first production line
constraint1=np.array([5,3,7,4,5])

#Unit raw material quantities of products for the second production line
constraint2=np.array([4,4,5,2,3])

# Model creating
model = pyo.ConcreteModel()

# decision varibles
model.X = pyo.Var(products, domain = pyo.NonNegativeReals) #the amount of production of the products to be produced
model.Y = pyo.Var(products, domain = pyo.Binary)   #products to be produced
model.K1=pyo.Var(domain = pyo.Binary)   #whether or not to produce eithor-or production constraint  
model.K2=pyo.Var(domain = pyo.Binary)   #whether or not to produce if-then production constraint

# Objective Function 
$Max = 50X_1+80X_2+70X_3+85X_4+90X_5-15000Y_1-35000Y_2-50000Y_3-25000Y_4-40000Y_5$

In [ ]:
#Objective Functuion 

model.obj_val = pyo.Objective(
    expr = sum(model.X[i]*profit[i] for i in products)-(sum(model.Y[i]*investment_cost[i] for i in products)),
    sense = pyo.maximize
)

The objective function indicates to maximize the total net profit of ATK-White company.

# Subject To 
$5X_1+3X_2+7X_3+4X_4+5X_5 \leq 7000$ \\
$4X_1+4X_2+5X_3+2X_4+3X_5 \leq 7000$ \\
$X_1 \leq 1400Y_1$ \\
$X_2 \leq 1750Y_2$ \\
$X_3 \leq 1000Y_3$ \\
$X_4 \leq 1750Y_4$ \\
$X_5 \leq 1000Y_1$ \\
$500-X_1-X_3 \leq 500K_1$ \\
$750-X_2-X_4 \leq 750(1-K_1)$ \\
$X_4-500 \leq 1750K_2$ \\
$X_5-500 \leq 1400(1-K_2)$ \\
$Y_1+Y_2+Y_3+Y_4+Y_5 \leq 3$ \\
$Y_3 \leq Y_1+Y_2$ \\
$Y_4 \leq Y_1+Y_2$ \\
$X_1,X_2,X_3,X_4,X_5 \geq 0$ \\
$Y_1,Y_2,Y_3,Y_4,Y_5 = 0$ or $1$ \\
$K_1,K_2 = 0$ or $1$ \\

In [ ]:
# Big numbers
M1=1400
M2=1750
M3=1000
M4=1750
M5=1400
M6=500
M7=750
M8=1750
M9=1400


# Constraint for labor 

model.labor_hours = pyo.Constraint(
    expr = sum(model.X[i]*constraint1[i] for i in products)<=7000
)


# Constraint for the raw material 

model.raw_material = pyo.Constraint(
    expr = sum(model.X[i]*constraint2[i] for i in products)<= 7000
)


#Constraint for when X1 takes a value greater than 0, its corresponding Y1 ≥ 0 (ie = 1) 
model.product1 = pyo.Constraint(
    expr = model.X[0]<=M1*model.Y[0]
)


#Constraint for when X2 takes a value greater than 0, its corresponding Y2 ≥ 0 (ie = 1) 

model.product2 = pyo.Constraint(
    expr = model.X[1]<=M2*model.Y[1]
)


#Constraint for when X3 takes a value greater than 0, its corresponding Y3 ≥ 0 (ie = 1) 

model.product3 = pyo.Constraint(
    expr = model.X[2]<=M3*model.Y[2]
)


#Constraint for when X4 takes a value greater than 0, its corresponding Y4 ≥ 0 (ie = 1) 

model.product4 = pyo.Constraint(
    expr = model.X[3]<=M4*model.Y[3]
)


#Constraint for when X5 takes a value greater than 0, its corresponding Y5 ≥ 0 (ie = 1). 

model.product5 = pyo.Constraint(
    expr = model.X[4]<=M5*model.Y[4]
)


# Constraint on ordering a total of 500 units from the 1st and 2nd products or 750 units in total from the 3rd and 4th products  

model.either_or1=pyo.Constraint(
    expr=500-model.X[0]-model.X[2]<=M6*model.K1
)

model.either_or2=pyo.Constraint(
    expr=750-model.X[1]-model.X[3]<=M7*(1-model.K1)
)


#Constraint of If the 5th product is produced more than 500, the 4th product can be produced in 500 units at the most 

model.if_then1=pyo.Constraint(
    expr=model.X[3]-500<=M8*model.K2
)

model.if_then2=pyo.Constraint(
    expr=model.X[4]-500<=M9*(1-model.K2)
)


#Constraint to produce no more than three products 

model.production_permit=pyo.Constraint(
    expr=sum(model.Y[i] for i in products)<=3
)


#Constraint to produce at least one of product 1 or product 2 in order to produce product 3 or product 4  

model.production_condition1=pyo.Constraint(
    expr=(model.Y[2]<=model.Y[0]+model.Y[1])
)

model.production_condition2=pyo.Constraint(
    expr=(model.Y[3]<=model.Y[0]+model.Y[1])
)

# Results 

According to the results obtained, ATK-White needs to produce 1400 units of the second product and 700 units of the fourth product in order to achieve maximum profit. In this case, the company will use all of its 7000 labor hours and 7000 units of raw materials during the planning period. In addition, regarding the agreement of the company with a customer, which required a total of at least 500 units of the first and third products or 750 units of the second and fourth products, the need can be met from production of 2100 units of the second and fourth products. Since there has been no production of the fifth product, the constraint that a maximum of 500 units of the fourth product should be produced has not been met. Furthermore, since only the second product is produced from the first and second products, the fourth product cen also be produced.

In [ ]:
opt = pyo.SolverFactory("cbc")
opt.solve(model)

print("Max Profit:", pyo.value(model.obj_val))
print("X1:",pyo.value(model.X[0]))
print("X2:",pyo.value(model.X[1]))
print("X3:",pyo.value(model.X[2]))
print("X4:",pyo.value(model.X[3]))
print("X5:",pyo.value(model.X[4]))
print("Y1:",pyo.value(model.Y[0]))
print("Y2:",pyo.value(model.Y[1]))
print("Y3:",pyo.value(model.Y[2]))
print("Y4:",pyo.value(model.Y[3]))
print("Y5:",pyo.value(model.Y[4]))
print("K1:",pyo.value(model.K1))
print("K2:",pyo.value(model.K2))

Max Profit: 111500.0
X1: 0.0
X2: 1400.0
X3: 0.0
X4: 700.0
X5: 0.0
Y1: 0.0
Y2: 1.0
Y3: 0.0
Y4: 1.0
Y5: 0.0
K1: 1.0
K2: 1.0
